In [1]:
import subprocess 
import json
import os
from web3 import Web3
from dotenv import load_dotenv
from eth_account import Account
from bit import  PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI

In [2]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
load_dotenv
mnemonic = os.getenv('mnemonic', "ready render hobby seed vendor credit dust slam balance option moon stomach")
print(mnemonic)

ready render hobby seed vendor credit dust slam balance option moon stomach


In [3]:
from constants import *

In [4]:
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys

In [5]:
coins = {"eth", "btc-test", "btc"}
numderive = 3

In [6]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [1]:
eth_priv_key = keys["eth"][0]['privkey']
btc_priv_key = keys['btc-test'][0]['privkey']


print(json.dumps(eth_priv_key, indent=5, sort_keys=True))
print(json.dumps(btc_priv_key, indent=5, sort_keys=True))
#print(json.dumps(keys, indent=4, sort_keys=True))

NameError: name 'keys' is not defined

In [8]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "1Hk26uv3aSo2WmKNPbeUGjNWPFDBLU2waW",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KzQ3o74J1R3MFuzvRNQWjckRz3AYZHgggJRbs3P7TH1ZoXJqKety",
            "pubkey": "0399c9a8f6933934bfe811fc524c0e8684f971819398a0581f20ddf6371db0bb18",
            "pubkeyhash": "b7a4d19972d97725ebe1d980ac875f4b41322ec5",
            "xprv": "xprvA4EtnVRNTKMCodewrh6ipRZg96w7tyJMntxXmtPitFkMn7Dwe6BRfeC2bnL8DRs96stL1Kn752eZJqfFX1Fs25uaRxnUSSShU91qzeg5gWK",
            "xpub": "xpub6HEFBzxGHguW27jQxidjBZWQh8mcJS2DA7t8aGoLSbHLeuZ6BdVgDSWWT5kbpKZxGjjA8qDfgYCyKgBeau5Qp48c5KC9dTaYjWd3jLfF8MG"
        },
        {
            "address": "16LsmsW5nshaMWz4XXE9qZjE1HHEEF61Ma",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L3PRGxRUzyYGDEnWsdtyunpa5DCfYcgKfBSCCXGEHofUyxYj7zc6",
            "pubkey": "03412c487c7a81706519d3236215f717b9134a39e0447e1a70de08fd65d83ab7e1",
            "pubkeyhash":

In [14]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_account = priv_key_to_account(ETH,eth_priv_key)
btc_account = priv_key_to_account(BTCTEST,btc_priv_key)

In [15]:
print(eth_account)
print(btc_account)

<PrivateKeyTestnet: mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD>


In [16]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient,amount):
    # YOUR CODE HERE
    global tx_data
    if coin.upper() == 'ETH':
        gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return tx_data
    
    if coin.upper() == 'BTCTEST':
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])


In [17]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient,amount):
    if coin.upper() == 'ETH':
        tx_eth = create_tx(coin, account, recipient, amount)
        sign = account.signTrasaction(tx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_btctest = create_tx(coin, account, recipient, amount)
        sign_tx_btctest = account.sign_transaction(tx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_tx_btctest)
        return sign_tx_btctest

In [27]:
create_tx(BTCTEST, btc_account, "miZgMxdGzSxCTpWazfD2KqhewoUvcQ6CC1",0.1)

In [28]:
w3.eth.getBalance("0x02194a55DDA1B029F70220D53f099F38c629ab36")

0

In [20]:
load_dotenv('.env')

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [21]:
private_key = os.getenv("privkey")

account_one = Account.from_key(private_key)

In [22]:
print(account_one.address)

0x66b0742F0766Ee9af00e1A70127a93f40184c945


In [23]:
w3.eth.getBalance("0x66b0742F0766Ee9af00e1A70127a93f40184c945")

904625697166532776746648320380374280103671755200316906558262375061821325312

In [24]:

# double check if  I am connected to blockchain. 
w3.isConnected()

True

In [25]:
create_tx(ETH,eth_account,"0x66b0742F0766Ee9af00e1A70127a93f40184c945", 1000)

{'to': '0x66b0742F0766Ee9af00e1A70127a93f40184c945',
 'from': '0x02194a55DDA1B029F70220D53f099F38c629ab36',
 'value': 1000,
 'gasPrice': 1000000000,
 'gas': 21000,
 'nonce': 0}

In [26]:
send_tx(ETH, eth_account,"0x66b0742F0766Ee9af00e1A70127a93f40184c945", 1000)

AttributeError: 'LocalAccount' object has no attribute 'signTrasaction'

In [ ]:
w3.eth.getBalance("0x66b0742F0766Ee9af00e1A70127a93f40184c945")